In [1]:
import pandas as pd
import os
os.chdir(r'../../..')
from input_configuration import *
from emme_configuration import *

%matplotlib inline

Basic Run Settings:

In [2]:
print "Base Year: " + str(base_year)
print "Model Year: " + str(model_year)
print "Scenario Name: " + scenario_name

Base Year: 2014
Model Year: 2040
Scenario Name: 2040_plan_update


In [5]:
# Emme config settings
print "Distance Pricing Used: " + str(add_distance_pricing)
if add_distance_pricing == True:
    for tod in ['am','md','pm','ev','ni']:
        print "Distance Pricing " + (tod).upper() + ": " + str(distance_rate_dict[tod])        

Distance Pricing Used: True
Distance Pricing AM: 13.5
Distance Pricing MD: 8.5
Distance Pricing PM: 13.5
Distance Pricing EV: 8.5
Distance Pricing NI: 8.5


In [6]:
# Emme config settings
print "HOT Tolls Used: " + str(add_hot_lane_tolls)
if add_distance_pricing == True:
    for tod in ['am','md','pm','ev','ni']:
        print "Distance Pricing " + (tod).upper() + ": " + str(hot_rate_dict[tod])  

HOT Tolls Used: True
Distance Pricing AM: 35
Distance Pricing MD: 10
Distance Pricing PM: 35
Distance Pricing EV: 10
Distance Pricing NI: 10


### Compare Inputs

In [17]:
comparison_runs = {'2040 old': r'C:\Users\Brice\Documents\temp_run_compare'}
run_name = os.getcwd().split('\\')[-1]

run_dir_dict = {
        'new_code': os.getcwd(), 
    }

# Add runs, if set in standard_summary_configuration.py
if len(comparison_runs.keys()) > 0:
    for comparison_name, comparison_dir in comparison_runs.iteritems():
        run_dir_dict[comparison_name] = comparison_dir

In [18]:
# Combine the input files

# CSV
input_file_list = {
    'parking ensemble': r'inputs\scenario\landuse\parking_gz.csv',
    'distribute_jblm_jobs': r'inputs\scenario\landuse\distribute_jblm_jobs.csv',
    'p_r_nodes': r'inputs\scenario\landuse\p_r_nodes.csv',
    'military': r'inputs\scenario\landuse\parcels_military.csv',
    'headways': r'inputs\scenario\networks\headways.csv',
    'transit_stops': r'inputs\scenario\networks\transit\transit_stops.csv'  
}

In [31]:
# More oddball inputs
# Space delimited
    
input_file_list = {
    'landuse': r'inputs\scenario\landuse\parcels_urbansim.txt',
    'buffered_parcels': r'outputs\landuse\buffered_parcels.txt'
}
        
for input_name, input_dir in input_file_list.iteritems():
    df = pd.DataFrame()
    print input_name
    for run_name, run_dir in run_dir_dict.iteritems():
        print run_name
        _df = pd.read_csv(os.path.join(run_dir, input_dir), sep=' ')
        _df['source'] = run_name
        _df['input'] = input_name
        df = df.append(_df)
               
    df.to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)        

landuse
2040 old
new_code
buffered_parcels
2040 old
new_code


In [26]:
df_dict = {
    'tazdata': pd.DataFrame(),
    'tazindex': pd.DataFrame(),
    'transit_fares': pd.DataFrame(),
    'tolls': pd.DataFrame(),
    'fare_zones': pd.DataFrame(),
    'delay': pd.DataFrame(),
    'pnr': pd.DataFrame()
          }
for run_name, run_dir in run_dir_dict.iteritems():
    print run_name
    input_name = 'tazdata'
    _df = pd.read_csv(r'inputs\scenario\landuse\tazdata.in', sep=' |:', skiprows=4)
    _df = _df.reset_index()
    _df.columns = ['TAZ','column','drop','hhemp']
    _df.drop('drop',axis=1,inplace=True)
    _df['source'] = run_name
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
    input_name = 'tazindex'
    _df = pd.read_csv(r'inputs\scenario\landuse\TAZIndex.txt', sep='\t')
    _df['source'] = run_name
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
    input_name = 'transit_fares'
    for tod in ['am','md']:
        _df = pd.read_csv(r'inputs\scenario\networks\fares\\' + tod + '_fares_farebox.in', sep=' |:', skiprows=2)
        _df.reset_index(inplace=True)
        _df.columns = ['origin','destination','drop','fare']
        _df.drop('drop',axis=1,inplace=True)
        _df['tod'] = tod
        _df['source'] = run_name
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
    input_name = 'tolls'
    for tod in ['am','md','pm','ev','ni']:
        _df = pd.read_csv(r'inputs\scenario\networks\tolls\\' + tod + '_roadway_tolls.in', delim_whitespace=True, header=None)
        # Need to add columns here
        _df['tod'] = tod
        _df['source'] = run_name
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
    input_name = 'fare_zones'
    _df = pd.read_csv(r'inputs\scenario\networks\fares\transit_fare_zones.grt', sep=' |:', skiprows=9)
    _df.reset_index(inplace=True)
    _df.columns = ['drop1','fare_zone','drop2','taz']
    _df.drop(['drop1','drop2'],axis=1,inplace=True)
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
#     input_name = 'delay'
#     for tod in ['am','md','pm','ev','ni']:
#         _df = pd.read_csv(r'inputs\scenario\networks\rdly\\' + tod + r'_rdly.txt', sep=' ')
#         _df.reset_index(inplace=True)
#         _df.drop('index',axis=1,inplace=True)
#         _df['tod'] = tod
#         _df['source'] = run_name
#     df_dict[input_name] = df_dict[input_name].append(_df)
#     df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)
    
    input_name = 'pnr'
    _df = pd.read_csv(r'inputs\scenario\networks\pnr_lot_capacities.in', sep=' |:', skiprows=1)
    _df.reset_index(inplace=True)
    _df.columns = ['drop','taz','capacity']
    _df.drop('drop',axis=1,inplace=True)
    _df['source'] = run_name
    df_dict[input_name] = df_dict[input_name].append(_df)
    df_dict[input_name].to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+input_name+'.csv', index=False)

2040 old
new_code


In [27]:
os.path.join(run_dir_dict[run_name],'outputs\\')

'Q:\\sc_2040_old_new_compare\\outputs\\'

In [ ]:
# Also need:
# trucks, supplementals, networks etc.

In [25]:
run_dir_dict[run_name]

'Q:\\sc_2040_old_new_compare'

In [24]:
coef_list = [
    'AutoOwnershipModel_psrcper1.F12',
    'EscortTourModeModel.F12',
#     'IndividualPersonDayPatternModel.F12',
#     'IntermediateStopGenerationModel.F12',
#     'IntermediateStopLocationModel_psrcper1.F12',
#     'OtherHomeBasedTourModeModel.F12',
    'OtherHomeBasedTourTimeModel.F12',
    'OtherTourDestinationModel.F12',
    'PayToParkAtWorkplaceModel_psrcper1.F12',
    'PersonExactNumberOfToursModel.F12',
    'SchoolLocationModel.F12',
    'SchoolTourModeModel.F12',
    'SchoolTourTimeModel.F12',
    'TransitPassOwnershipModel.F12',
    'TripModeModel.F12',
    'TripTimeModel.F12',
    'WorkBasedSubtourGenerationModel.F12',
    'WorkBasedSubtourModeModel.F12',
    'WorkBasedSubtourTimeModel.F12',
    'WorkLocationModel.F12',
    'WorkTourDestinationModel_psrcper1.F12',
    'WorkTourModeModel.F12',
    'WorkTourTimeModel.F12'
]

df = pd.DataFrame()


for run_name, run_dir in run_dir_dict.iteritems():
    for coef in coef_list:
        print coef
        # Model inputs also are useful - these shouldn't change often, but may still want a point of comparison
        _df = pd.read_csv(r'inputs\model\coefficients\\' + coef, sep=' ', skipinitialspace=True, 
                         skiprows=2, error_bad_lines=False)
        _df.reset_index(inplace=True)
        _df.columns = ['var_number','description', 'suppressed', 'coefficient_value', 'significance', 'drop']
        _df.drop('drop', axis=1, inplace=True)
        # Coefficient list stops after -1 in var_number field; trim off other part 
        _df = _df.iloc[0:_df[_df['var_number'] == -1].index[0]]
#         results[coef] = df
        _df['file_name'] = coef
        _df['source'] = run_name
        df = df.append(_df)
        
df.to_csv(os.path.join(run_dir_dict[run_name],'outputs\\')+'coefficients.csv', index=False)

AutoOwnershipModel_psrcper1.F12
EscortTourModeModel.F12
OtherHomeBasedTourTimeModel.F12
OtherTourDestinationModel.F12


Skipping line 72: expected 6 fields, saw 10
Skipping line 73: expected 6 fields, saw 10
Skipping line 74: expected 6 fields, saw 10
Skipping line 75: expected 6 fields, saw 10
Skipping line 76: expected 6 fields, saw 10
Skipping line 77: expected 6 fields, saw 10
Skipping line 78: expected 6 fields, saw 10
Skipping line 79: expected 6 fields, saw 10
Skipping line 80: expected 6 fields, saw 10
Skipping line 81: expected 6 fields, saw 10
Skipping line 82: expected 6 fields, saw 10
Skipping line 83: expected 6 fields, saw 10
Skipping line 84: expected 6 fields, saw 10
Skipping line 85: expected 6 fields, saw 10
Skipping line 86: expected 6 fields, saw 10
Skipping line 87: expected 6 fields, saw 10
Skipping line 88: expected 6 fields, saw 10
Skipping line 89: expected 6 fields, saw 10
Skipping line 90: expected 6 fields, saw 10
Skipping line 91: expected 6 fields, saw 10
Skipping line 92: expected 6 fields, saw 10
Skipping line 93: expected 6 fields, saw 10
Skipping line 94: expected 6 fie

PayToParkAtWorkplaceModel_psrcper1.F12
PersonExactNumberOfToursModel.F12



Skipping line 16: expected 6 fields, saw 7
Skipping line 17: expected 6 fields, saw 10
Skipping line 18: expected 6 fields, saw 10
Skipping line 19: expected 6 fields, saw 10
Skipping line 20: expected 6 fields, saw 10

Skipping line 302: expected 6 fields, saw 10
Skipping line 303: expected 6 fields, saw 10
Skipping line 304: expected 6 fields, saw 10
Skipping line 305: expected 6 fields, saw 10
Skipping line 306: expected 6 fields, saw 10
Skipping line 307: expected 6 fields, saw 10
Skipping line 308: expected 6 fields, saw 10
Skipping line 309: expected 6 fields, saw 10
Skipping line 310: expected 6 fields, saw 10
Skipping line 311: expected 6 fields, saw 10
Skipping line 312: expected 6 fields, saw 10
Skipping line 313: expected 6 fields, saw 10
Skipping line 314: expected 6 fields, saw 10
Skipping line 315: expected 6 fields, saw 10
Skipping line 316: expected 6 fields, saw 10
Skipping line 317: expected 6 fields, saw 10
Skipping line 318: expected 6 fields, saw 10
Skipping line 

SchoolLocationModel.F12
SchoolTourModeModel.F12
SchoolTourTimeModel.F12
TransitPassOwnershipModel.F12

...

SchoolLocationModel.F12
SchoolTourModeModel.F12
SchoolTourTimeModel.F12
TransitPassOwnershipModel.F12


Skipping line 32: expected 6 fields, saw 10
Skipping line 33: expected 6 fields, saw 10
Skipping line 34: expected 6 fields, saw 10
Skipping line 35: expected 6 fields, saw 10
Skipping line 36: expected 6 fields, saw 10
Skipping line 37: expected 6 fields, saw 10
Skipping line 38: expected 6 fields, saw 10
Skipping line 39: expected 6 fields, saw 10
Skipping line 40: expected 6 fields, saw 10
Skipping line 41: expected 6 fields, saw 10
Skipping line 42: expected 6 fields, saw 10
Skipping line 43: expected 6 fields, saw 10
Skipping line 44: expected 6 fields, saw 10
Skipping line 45: expected 6 fields, saw 10
Skipping line 46: expected 6 fields, saw 10
Skipping line 47: expected 6 fields, saw 10
Skipping line 48: expected 6 fields, saw 10
Skipping line 49: expected 6 fields, saw 10
Skipping line 50: expected 6 fields, saw 10
Skipping line 51: expected 6 fields, saw 10
Skipping line 52: expected 6 fields, saw 10
Skipping line 53: expected 6 fields, saw 10
Skipping line 54: expected 6 fie

TripModeModel.F12
TripTimeModel.F12
WorkBasedSubtourGenerationModel.F12
WorkBasedSubtourModeModel.F12
WorkBasedSubtourTimeModel.F12


Skipping line 57: expected 6 fields, saw 10
Skipping line 58: expected 6 fields, saw 10
Skipping line 59: expected 6 fields, saw 10
Skipping line 60: expected 6 fields, saw 10
Skipping line 61: expected 6 fields, saw 10
Skipping line 62: expected 6 fields, saw 10
Skipping line 63: expected 6 fields, saw 10
Skipping line 64: expected 6 fields, saw 10
Skipping line 65: expected 6 fields, saw 10
Skipping line 66: expected 6 fields, saw 10
Skipping line 67: expected 6 fields, saw 10
Skipping line 68: expected 6 fields, saw 10
Skipping line 69: expected 6 fields, saw 10
Skipping line 70: expected 6 fields, saw 10
Skipping line 71: expected 6 fields, saw 10
Skipping line 72: expected 6 fields, saw 10
Skipping line 73: expected 6 fields, saw 10
Skipping line 74: expected 6 fields, saw 10
Skipping line 75: expected 6 fields, saw 10
Skipping line 76: expected 6 fields, saw 10
Skipping line 77: expected 6 fields, saw 10
Skipping line 78: expected 6 fields, saw 10
Skipping line 79: expected 6 fie

WorkLocationModel.F12
WorkTourDestinationModel_psrcper1.F12
WorkTourModeModel.F12
WorkTourTimeModel.F12


In [354]:
df.groupby('file_name').sum()

,var_number,coefficient_value
file_name,,
AutoOwnershipModel_psrcper1.F12,5296,-81.597217
EscortTourModeModel.F12,3510,-51.275902
OtherHomeBasedTourTimeModel.F12,15358,-449.731504
OtherTourDestinationModel.F12,11594,-1062.524789
PayToParkAtWorkplaceModel_psrcper1.F12,114,-11.741030
PersonExactNumberOfToursModel.F12,249832,-166.066796
SchoolLocationModel.F12,5374,-704.300995
SchoolTourModeModel.F12,6206,-24.572492
SchoolTourTimeModel.F12,6814,-448.658529


In [247]:
# Model inputs also are useful - these shouldn't change often, but may still want a point of comparison
df = pd.read_csv(r'inputs\model\coefficients\EscortTourModeModel.F12', sep=' ', skipinitialspace=True, 
                 skiprows=2, error_bad_lines=False)
df.reset_index(inplace=True)
df.columns = ['var_number','description', 'suppressed', 'coefficient_value', 'significance', 'drop']
df
df.drop('drop', axis=1, inplace=True)
# Coefficient list stops after -1 in var_number field; trim off other part 
df = df.iloc[0:df[df['var_number'] == -1].index[0]]
df

Skipping line 23: expected 6 fields, saw 10
Skipping line 24: expected 6 fields, saw 10
Skipping line 25: expected 6 fields, saw 10
Skipping line 26: expected 6 fields, saw 10
Skipping line 27: expected 6 fields, saw 10
Skipping line 28: expected 6 fields, saw 10
Skipping line 29: expected 6 fields, saw 10
Skipping line 30: expected 6 fields, saw 10
Skipping line 31: expected 6 fields, saw 10
Skipping line 32: expected 6 fields, saw 10
Skipping line 33: expected 6 fields, saw 10
Skipping line 34: expected 6 fields, saw 10



,var_number,description,suppressed,coefficient_value,significance
0,2,timeutil,F,2.594432,.121537711752
1,30,s3-const,F,-0.989365,.319545518056
2,31,s3-hhcu5,F,1.335395,.757384490262E-01
3,32,s3-hh515,F,0.529875,.417772373411E-01
4,33,s3-hhdas,F,0.276831,.106871305268
5,40,s2-const,F,-0.125647,.322420893109
6,41,sr-nocars,T,-5.914000,.000000000000
7,42,sr-carsltd,F,0.017161,.102197155318
8,50,hov-const,T,-10.000000,.000000000000
9,60,bi-const,T,-10.000000,.000000000000


In [ ]:
# Intrazonals

In [ ]:
# Terminal Times

In [ ]:
# TAZ Acres

In [ ]:
# Roster?

In [ ]:
# VDFs?

### Landuse Inputs

In [25]:
# Landuse
df = pd.read_csv(r'inputs/scenario/landuse/parcels_urbansim.txt',sep=' ')

In [46]:
# Daily Parking Price overall
df['PARKDY_P'].agg({'min': 'min',' max': 'max', 'mean': 'mean'})


mean     16.178937
 max    195.480734
min       0.000000
Name: PARKDY_P, dtype: float64

In [47]:
# Daily Parking Price overall
df['PPRICDYP'].agg({'min': 'min',' max': 'max', 'mean': 'mean'})

mean      81.73359
 max    2898.00000
min        0.00000
Name: PPRICDYP, dtype: float64

In [49]:
df['PARKDY_P'].agg({'min': 'min',' max': 'max', 'mean': 'mean'})

mean     16.178937
 max    195.480734
min       0.000000
Name: PARKDY_P, dtype: float64

In [41]:
# Hourly Parking Price overall (in )
df['PPRICHRP'].agg({'min': 'min',' max': 'max', 'mean': 'mean'})


mean     14.534554
 max    198.278351
min       0.000000
Name: PARKHR_P, dtype: float64

In [42]:

df['EMPEDU_P'].agg({'total':'sum','min': 'min',' max': 'max', 'mean': 'mean'})

mean          0.150697
total    192348.000000
 max       4860.000000
min           0.000000
Name: EMPEDU_P, dtype: float64

In [43]:

df['HH_P'].agg({'total':'sum','min': 'min',' max': 'max', 'mean': 'mean'})

mean     1.651433e+00
total    2.107868e+06
 max     5.523000e+03
min      0.000000e+00
Name: HH_P, dtype: float64

In [44]:

df['NPARKS'].agg({'total':'sum','min': 'min',' max': 'max', 'mean': 'mean'})

mean        0.007019
total    8959.000000
 max        1.000000
min         0.000000
Name: NPARKS, dtype: float64

In [45]:

df['SQFT_P'].agg({'total':'sum','min': 'min',' max': 'max', 'mean': 'mean'})

mean     6.147507e+04
total    7.846598e+10
 max     7.412599e+07
min      0.000000e+00
Name: SQFT_P, dtype: float64

In [40]:
df.columns

Index([u'APARKS', u'EMPEDU_P', u'EMPFOO_P', u'EMPGOV_P', u'EMPIND_P',
       u'EMPMED_P', u'EMPOFC_P', u'EMPOTH_P', u'EMPRET_P', u'EMPRSC_P',
       u'EMPSVC_P', u'EMPTOT_P', u'HH_P', u'LUTYPE_P', u'NPARKS', u'PARCELID',
       u'SQFT_P', u'STUGRD_P', u'STUHGH_P', u'STUUNI_P', u'TAZ_P', u'XCOORD_P',
       u'YCOORD_P', u'PPRICHRP', u'PPRICDYP', u'PARKDY_P', u'PARKHR_P'],
      dtype='object')

### Accessibility

In [22]:
# Bike Inputs
df = pd.read_csv(r'inputs/scenario/bike/bike_attributes.csv', sep=' ')
print 'Bike Facility Type (mean): ' + str(df['@bkfac'].mean())
print 'Bike Facility Upslope (mean): ' + str(df['@upslp'].mean())

Bike Facility Type (mean): 0.634752208047
Bike Facility Upslope (mean): 0.0332534048304


In [21]:
# Compare inputs between two runs
df

,inode,jnode,@bkfac,@upslp
0,186147,58936,0,0.006313
1,73585,73602,8,0.075141
2,58182,58072,0,0.041988
3,88437,88390,0,0.014121
4,89901,183605,0,0.001109
5,198189,106200,0,0.689359
6,186553,186551,0,1.414597
7,104735,102125,0,0.000000
8,96007,89923,0,1.767392
9,81520,81585,0,0.001600
